***구글 드라이브 마운트**

*   구글 드라이브에 있는 파일을 불러와서 사용하기 위해서는 드라이브 마운트를 해야 한다.


1.   코드를 실행하여 구글 드라이브 인증 진행
2.   링크로 이동하여 Google Drive File Stream 계정 엑세스 요청 허가하기







In [1]:
import pandas as pd

# 구글 드라이브와 연동
from google.colab import drive
drive.mount('/content/gdrive')    # 이후 파일 업로드에 사용하는 경로는 '/content/gdrive/My Drive/' 이 됨


Mounted at /content/gdrive


##**[1단계] 감성대화말뭉치 원천 데이터를 기본으로 전처리 시작**


**사용하는 파일 : 원천데이터 - 감성대화말뭉치(최종데이터)_Training.csv** (사람문장 1만 활용)

In [40]:
df = pd.read_csv('/content/gdrive/My Drive/sentiment/data.csv', encoding='cp949')
df = df[['sentiment','sentence']]
print(set(df['sentiment']))

{'분노', '당황', '기쁨', '슬픔', '중성', '불안'}


### **사용하는 파일 : 감정단어사전 활용**
감정분류 5개 버전
**dict_5.csv**

In [41]:
dict_5 = pd.read_csv('/content/gdrive/My Drive/sentiment/dict_5.csv', encoding='utf-8')
dict_5 = dict_5.dropna(axis=0)

col_name2=['감정범주', '단어']
dict_5 = dict_5[col_name2]    

In [42]:
df2 = pd.DataFrame({
    'sentiment' : df['sentiment'].tolist()+dict_5['감정범주'].tolist(),
    'sentence' : df['sentence'].tolist()+dict_5['단어'].tolist()
})

**결측치 제거**

In [7]:
print("결측치 제거 전 행 개수")
print(df2.size)
df2 = df2.dropna(axis=0) # 결측치 제거
print("결측치 제거 후 행 개수")
print(df2.size)

결측치 제거 전 행 개수
82964
결측치 제거 후 행 개수
82964


##**[2단계] 분석대상 컬럼 추출 + 감정 라벨링**

**감정 라벨링**

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [43]:
# 범주 찾기
le.fit(df2['sentiment'])
# 범주형 -> 수치형 데이터로 변환
label_encoded = le.transform(df2['sentiment'])
# 기쁨(0), 당황(1), 분노(2), 불안(3), 슬픔(4), 중성(5)

In [54]:
# sentiment 컬럼에 넣기
df2['sentiment'] = label_encoded
df2.head()

,sentiment,sentence
0,2,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,2,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,2,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분해.
4,2,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.


##**[3단계] 텍스트 정규화**
토큰화 : 문서를 문장 다뉘로, 문장을 단어 단위로 쪼개는 것. 주로 N-gram 방법이 사용됨
필터링, 불용어 제거, 철자 수정 : 텍스트로부터 주요한 정보를 얻는 것에 영향을 미치지 않는 단어들을 제거한다.
어근 추출(Stemming/Lemmatization)

**토큰화**

In [13]:
import nltk
from nltk import word_tokenize, sent_tokenize

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 36.6 MB/s eta 0:00:00


**형태소 추출**

In [16]:
from konlpy.tag import Okt
okt = Okt()

**불용어 처리**

In [17]:
stop_words = [
"나","내", "과", "무엇", "거", "있다","요즘","다","날","되다","있다","노","오늘", "김", "그래서", "우리","저희","따라","의해","에게","뿐","의거","하여","근거","입각","기준","예","하면","들면","들자면","저","소인","소생","저희","다른","물론","또한","그리고","비길","수",
"없다","해서는","막론","비록","더라도","하다","만","하는","편이","불문","향","하여","쪽","틈타","어떻다", "달","타다","오르다","것", "얼마","제외","이","외","밖","하여야",
"한다면","이", "곳","여기","부터","기점","따라서","할","생각","하려고","하다","함", "때","앞","중","보는데서","해야","일것이다","할줄","알다","등","등등","제","겨우","단지",
"다만","할","딩동","대해","대하","얼마나","얼마만큼","큼","얼","남짓","남자","여자","남","여","녀","얼마간","약간","다소","좀","조금","다수","몇","하물며","또한","이외",
"대해","말","하자면","다음","이","저","그","바꾸어서","말","만약","툭","각","각각","여러분","각종","각자","제각기","하도록","까닭","관","하여","진짜",
"무슨","어디","곳","더군다나","언제","또","혹시","젠","되어","및","이번","다음","된","거의","이제","감","점점","사람","같다","전","후","데","매일","받다","난","왜",
"왼쪽", "오른쪽", "오늘날", "친구", "두", "해", "나다"]

**한국어 제외 불용어 제거하는 함수 생성**

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [18]:
import re
def cleaning(ary):
  for idx, line in  enumerate(ary):
    words = []
    # 형태소 추출
    for word in okt.pos(line, stem=True):
        if word[1] in ['Noun', 'Verb', 'Adjective', 'Adverb','KoreanParticle','Exclamation']:
          word = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',word[0])
          if word != '':
            if word not in stop_words:
              words.append(word)
          # # 영어 제거
          # word = re.sub('([a-zA-Z])','',word[0])   
          # # 자음 모음 제거
          # word = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',word[0])
          # # 특수문자 제거
          # word = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',word[0])
    ary[idx] = ' '.join(words) 
  return ary


**텍스트 정규화**

In [19]:
sentence = cleaning(df2['sentence'])

<ipython-input-18-41422fec4e01>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ary[idx] = ' '.join(words)


**정규화 된 sentence로 데이터프레임 값 변경**

In [21]:
df3 = pd.DataFrame({
    'sentiment' : df2['sentiment'].tolist(),
    'sentence' : sentence
})
df3.head()

,sentiment,sentence
0,2,일 해도 해도 끝 화가
1,2,급여 깎다 물가 월급 자꾸 깎다 너무 화가
2,2,회사 신입 들어오다 말투 거슬리다 그렇다 애 보다 스트레스
3,2,직장 막내 이유 온갖 심부름 시키다 일도 많다 정말 분해
4,2,입사 신입사원 나르다 무시 너무 화가


**csv로 변환해서 내보내기**

In [23]:
df3.to_csv("data_clean.csv",encoding='CP949')

##**[4단계] 피처 벡터화**
텍스트를 숫자로 변형해주는 과정
CountVectorizer 방법 : 단어의 빈도수에 기반

**train, test 데이터 분리 및 피처 벡터화**

In [24]:
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer   # tf-idf 방식을 사용하려면 대신 TfidfVectorizer를 import
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [25]:
y_target = df3["sentiment"]
X_feature = df3["sentence"]

In [26]:
vect = CountVectorizer(min_df = 5).fit(X_feature)
X_feature = vect.transform(X_feature)

In [27]:
X_train, X_test, y_train, y_test= train_test_split(X_feature, y_target, test_size=0.2, random_state=156)

X_train.shape, X_test.shape

((33185, 3734), (8297, 3734))

**countVectorizer 결과 저장**

In [36]:
import pickle

with open('vect.pkl', 'wb') as f:  # cv.pkl이라는 파일을 바이너리 쓰기(wb)모드로 열어서 f라 하고
    pickle.dump(vect, f)           # cv를 f에 저장

#**[5단계]모델 학습**

**[로지스틱 회귀]**

In [28]:
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=5000)

In [29]:
y_predict = model.predict(X_test)

In [30]:
print("테스트 데이터 점수:", model.score(X_test, y_test))
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_predict))) 

테스트 데이터 점수: 0.8253585633361455
Accuracy: 0.83


**모델 저장**

In [32]:
import joblib

In [33]:
joblib.dump(model, 'model_logistic.pb') 

['model_logistic.pb']



---



**[랜덤 포레스트]** 

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

random_forest = RandomForestClassifier(random_state=32)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83      1350
           1       0.86      0.58      0.69       689
           2       0.81      0.70      0.75      1564
           3       0.90      0.79      0.84       787
           4       0.79      0.90      0.84      3660
           5       0.95      0.90      0.92       247

    accuracy                           0.81      8297
   macro avg       0.86      0.78      0.81      8297
weighted avg       0.82      0.81      0.81      8297



**모델 저장**

In [35]:
joblib.dump(model, 'model_random_forest.pb') 

['model_random_forest.pb']